### Individual Income Prediction Using Decision Tree

#### In this project, we'll be looking at individual income in the United States. The data is from the 1994 census, and contains information on an individual's marital status, age, type of work, and more. The target column, or what we want to predict, is whether individuals make less than or equal to 50k a year, or more than 50k a year. The data from the University of California, Irvine's website.

In [1]:
# import libraries and data
import pandas
# Set index_col to False to avoid pandas thinking that the first column is row indexes (it's age)
income = pandas.read_csv("income.csv", index_col=False)
income.head(5)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [2]:
income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
high_income       32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


### Convert categorical data into intgeter

In [3]:
income['marital_status'][0]

' Never-married'

In [4]:
marital_mapper = {' Married-civ-spouse': 0,
                 ' Never-married': 1,
                 ' Divorced': 2,
                 ' Separated': 3,
                 ' Widowed': 4,
                 ' Married-spouse-absent': 5,
                 ' Married-AF-spouse': 6}
income['marital_status'] = income['marital_status'].map(marital_mapper)
income['marital_status'].value_counts()

0    14976
1    10683
2     4443
3     1025
4      993
5      418
6       23
Name: marital_status, dtype: int64

In [5]:
workclass_mapper = {' Private': 0,
                   ' Self-emp-not-inc': 1,
                   ' Local-gov': 2,
                   ' ?': 3,
                   ' State-gov': 4,
                   ' Self-emp-inc': 5,
                   ' Federal-gov': 6,
                   ' Without-pay': 7,
                   ' Never-worked': 8}
income['workclass'] = income['workclass'].map(workclass_mapper)
income['workclass'].value_counts()

0    22696
1     2541
2     2093
3     1836
4     1298
5     1116
6      960
7       14
8        7
Name: workclass, dtype: int64

In [6]:
occupation_mapper = {' Prof-specialty': 0,
                     ' Craft-repair': 1,     
                     ' Exec-managerial': 2,  
                     ' Adm-clerical': 3,     
                     ' Sales': 4,            
                     ' Other-service':5,    
                     ' Machine-op-inspct':6,
                     ' ?': 7,                
                     ' Transport-moving': 8, 
                     ' Handlers-cleaners': 9,
                     ' Farming-fishing': 10,  
                     ' Tech-support': 11,     
                     ' Protective-serv': 12,  
                     ' Priv-house-serv': 13,  
                     ' Armed-Forces': 14}
income['occupation'] = income['occupation'].map(occupation_mapper)
income['occupation'].value_counts()

0     4140
1     4099
2     4066
3     3770
4     3650
5     3295
6     2002
7     1843
8     1597
9     1370
10     994
11     928
12     649
13     149
14       9
Name: occupation, dtype: int64

In [7]:
relationship_mapper = { ' Husband': 0,
                         ' Not-in-family': 1,
                         ' Own-child': 2,
                         ' Unmarried': 3,
                         ' Wife': 4,
                         ' Other-relative': 5}
income['relationship'] = income['relationship'].map(relationship_mapper)
income['relationship'].value_counts()

0    13193
1     8305
2     5068
3     3446
4     1568
5      981
Name: relationship, dtype: int64

In [8]:
race_mapper = {  ' White': 0,
                 ' Black': 1,
                 ' Asian-Pac-Islander': 2,
                 ' Amer-Indian-Eskimo': 3,
                 ' Other'             : 4}
income['race'] = income['race'].map(race_mapper)
income['race'].value_counts()

0    27816
1     3124
2     1039
3      311
4      271
Name: race, dtype: int64

In [9]:
sex_mapper = {' Male': 0, ' Female': 1}
income['sex'] = income['sex'].map(sex_mapper)
income['sex'].value_counts()

0    21790
1    10771
Name: sex, dtype: int64

In [10]:
income['high_income'][0:10]

0     <=50K
1     <=50K
2     <=50K
3     <=50K
4     <=50K
5     <=50K
6     <=50K
7      >50K
8      >50K
9      >50K
Name: high_income, dtype: object

In [11]:
high_income_mapper = {' >50K': 1, ' <=50K': 0}
income['high_income'] = income['high_income'].map(high_income_mapper)
income['high_income'].value_counts()

0    24720
1     7841
Name: high_income, dtype: int64

In [12]:
native_country_counts = income['native_country'].value_counts()
native_country_mapper = {}
for i, name in enumerate(native_country_counts.index):
    native_country_mapper[name] = i
print(native_country_mapper)
income['native_country'] = income['native_country'].map(native_country_mapper)
income['native_country'].value_counts()

{' United-States': 0, ' Mexico': 1, ' ?': 2, ' Philippines': 3, ' Germany': 4, ' Canada': 5, ' Puerto-Rico': 6, ' El-Salvador': 7, ' India': 8, ' Cuba': 9, ' England': 10, ' Jamaica': 11, ' South': 12, ' China': 13, ' Italy': 14, ' Dominican-Republic': 15, ' Vietnam': 16, ' Guatemala': 17, ' Japan': 18, ' Poland': 19, ' Columbia': 20, ' Taiwan': 21, ' Haiti': 22, ' Iran': 23, ' Portugal': 24, ' Nicaragua': 25, ' Peru': 26, ' Greece': 27, ' France': 28, ' Ecuador': 29, ' Ireland': 30, ' Hong': 31, ' Cambodia': 32, ' Trinadad&Tobago': 33, ' Thailand': 34, ' Laos': 35, ' Yugoslavia': 36, ' Outlying-US(Guam-USVI-etc)': 37, ' Honduras': 38, ' Hungary': 39, ' Scotland': 40, ' Holand-Netherlands': 41}


0     29170
1       643
2       583
3       198
4       137
5       121
6       114
7       106
8       100
9        95
10       90
11       81
12       80
13       75
14       73
15       70
16       67
17       64
18       62
19       60
20       59
21       51
22       44
23       43
24       37
25       34
26       31
27       29
28       29
29       28
30       24
31       20
33       19
32       19
35       18
34       18
36       16
37       14
39       13
38       13
40       12
41        1
Name: native_country, dtype: int64

In [13]:
income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null int64
fnlwgt            32561 non-null int64
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null int64
occupation        32561 non-null int64
relationship      32561 non-null int64
race              32561 non-null int64
sex               32561 non-null int64
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null int64
high_income       32561 non-null int64
dtypes: int64(14), object(1)
memory usage: 3.7+ MB


#### We can split the data set into two portions based on whether the individual works in the private sector or not.

In [14]:
# private_incomes should contain all rows where workclass is 4.
# public_incomes should contain all rows where workclass is not 4.
private_incomes = income[income["workclass"] == 4]
public_incomes = income[income["workclass"] != 4]
print(private_incomes.shape)
print(public_incomes.shape)

(1298, 15)
(31263, 15)


#### When we performed the split, 9865 rows went to the left, where workclass does not equal 4, and 22696 rows went to the right, where workclass equals 4.

#### It's useful to think of a decision tree as a flow of rows of data. When we make a split, some rows will go to the right, and some will go to the left. As we build the tree deeper and deeper, each node will "receive" fewer and fewer rows.

#### The nodes at the bottom of the tree, where we decide to stop splitting, are called terminal nodes, or leaves. When we do our splits, we aren't doing them randomly; we have an objective. Our goal is to ensure that we can make a prediction on future data. In order to do this, all rows in each leaf must have only one value for our target column.

#### We're trying to predict the high_income column.
- If high_income is 1, it means that the person has an income higher than 50k a year.
- If high_income is 0, it means that they have an income less than or equal to 50k a year.

#### After constructing a tree using the income data, we'll want to make predictions. In order to do this, we'll take a new row and feed it through our decision tree.

#### First, we check whether the person works in the private sector. If they do, we'll check to see whether they're native to the US, and so on.

#### Eventually, we'll reach a leaf. The leaf will tell us what value we should predict for high_income.

#### In order to be able to make this prediction, all of the rows in a leaf should only have a single value for high_income. This means that leaves can't have both 0 and 1 values in the high_income column. Each leaf can only have rows with the same values for our target column. If this isn't the case, we won't be able to make effective predictions.

#### We'll need to continue splitting nodes until we get to a point where all of the rows in a node have the same value for high_income.

#### Now that we have a high-level view of how decision trees work, let's explore the details and learn how to perform the splits.

#### We'll use a specific measure to figure out which variables we should split nodes on. Post-split, we'll have two data sets, each containing the rows from one branch of the split.

#### Because we're trying to reach the leaves having only 1s or only 0s in high_income, each split will need to get us closer to that goal.

#### When we split, we'll try to separate as many 0s from 1s in the high_income column as we can. In order to do this, we need a metric for how "together" the different values in the high_income column are.

#### Data scientists commonly use a metric called entropy for this purpose. Entropy refers to disorder. The more "mixed together" 1s and 0s are, the higher the entropy. A data set consisting entirely of 1s in the high_income column would have low entropy.

#### Entropy, which is not to be confused with entropy from physics, comes from information theory. Information theory is based on probability and statistics, and deals with the transmission, processing, utilization, and extraction of information. A key concept in information theory is the notion of a bit of information. One bit of information is one unit of information.

#### We can represent a bit of information as a binary number because it either has the value 1 or 0. Suppose there's an equal probability of tomorrow being sunny (1) or not sunny (0). If I tell you that it will be sunny, I've given you one bit of information.

#### We can also think of entropy in terms of information. If we flip a coin where both sides are heads, we know upfront that the result will be heads. We gain no new information by flipping the coin, so entropy is 0. On the other hand, if the coin has a heads side and a tails side, there's a 50% probability that it will land on either. Thus, flipping the coin gives us one bit of information -- which side the coin landed on.

### Compute entropy

#### Compute the entropy of the high_income column in the income dataframe, and assign the result to income_entropy

In [15]:
import math
# We'll do the same calculation we did above, but in Python
# Passing in 2 as the second parameter to math.log will take a base 2 log
entropy = -(2/5 * math.log(2/5, 2) + 3/5 * math.log(3/5, 2))
print(entropy)
prob_0 = income[income["high_income"] == 0].shape[0] / income.shape[0]
prob_1 = income[income["high_income"] == 1].shape[0] / income.shape[0]
print('p_0: {}'.format(prob_0))
print('p_1: {}'.format(prob_1))
income_entropy = -(prob_0 * math.log(prob_0, 2) + prob_1 * math.log(prob_1, 2))
income_entropy

0.9709505944546686
p_0: 0.7591904425539756
p_1: 0.2408095574460244


0.7963839552022132

#### We'll need a way to go from computing entropy to figuring out which variable to split on. We can do this using information gain, which tells us which split will reduce entropy the most.

#### We're computing information gain () for a given target variable (), as well as a given variable we want to split on ().

#### To compute it, we first calculate the entropy for . Then, for each unique value  in the variable , we compute the number of rows in which  takes on the value , and divide it by the total number of rows. Next, we multiply the results by the entropy of the rows where  is . We add all of these subset entropies together, then subtract from the overall entropy to get information gain.

#### Here's an alternate explanation. We're finding the entropy of each set post-split, weighting it by the number of items in each split, then subtracting from the current entropy. If the result is positive, we've lowered entropy with our split. The higher the result is, the more we've lowered entropy.

#### One strategy for constructing trees is to create as many branches at each node as there are unique values for the variable we're splitting on. So if the variable has three or four values, we'd end up with three or four branches. This approach usually involves more complexity than it's worth and doesn't improve prediction accuracy, but it's worth knowing about.

#### To simplify the calculation of information gain and make splits simpler, we won't do it for each unique value. We'll find the median for the variable we're splitting on instead. Any rows where the value of the variable is below the median will go to the left branch, and the rest of the rows will go to the right branch. To compute information gain, we'll only have to compute entropies for two subsets.

### Compute information gain

#### Compute the information gain for splitting on the age column of income.

In [16]:
# compute the median of age.
# assign anything less than or equal to the median to 
# the left branch, and anything greater than the median 
# to the right branch.
# compute the information gain and assign it to age_information_gain
import numpy

def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array.
    """
    # Compute the counts of each unique value in the column
    counts = numpy.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    
    return -entropy

# Verify that our function matches our answer from earlier
entropy = calc_entropy([1,1,0,0,1])
print(entropy)

information_gain = entropy - ((.8 * calc_entropy([1,1,0,0])) + (.2 * calc_entropy([1])))
print(information_gain)
income_entropy = calc_entropy(income["high_income"])

median_age = income["age"].median()

left_split = income[income["age"] <= median_age]
right_split = income[income["age"] > median_age]

age_information_gain = income_entropy - ((left_split.shape[0] / income.shape[0]) * calc_entropy(left_split["high_income"]) + ((right_split.shape[0] / income.shape[0]) * calc_entropy(right_split["high_income"])))

0.9709505944546686
0.17095059445466854


#### Now that we know how to compute information gain, we can determine the best variable to split a node on. When we start our tree, we want to make an initial split. We'll find the variable to split on by calculating which split would have the highest information gain.

#### Create a list called information_gains. It should contain, in order, the information gain from splitting on these columns: age, workclass, education_num, marital_status, occupation, relationship, race, sex, hours_per_week, native_country.

#### Find the highest value in the information_gains list, and assign the corresponding column name to highest_gain

In [18]:
def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on, and target
    """
    # Calculate the original entropy
    original_entropy = calc_entropy(data[target_name])
    
    # Find the median of the column we're splitting
    column = data[split_name]
    median = column.median()
    
    # Make two subsets of the data, based on the median
    left_split = data[column <= median]
    right_split = data[column > median]
    
    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * calc_entropy(subset[target_name])
    
    # Return information gain
    return original_entropy - to_subtract

# Verify that our answer is the same as on the last screen
print(calc_information_gain(income, "age", "high_income"))

columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]
information_gains = []
# Loop through and compute information gains
for col in columns:
    information_gain = calc_information_gain(income, col, "high_income")
    information_gains.append(information_gain)

# Find the name of the column with the highest gain
highest_gain_index = information_gains.index(max(information_gains))
highest_gain = columns[highest_gain_index]
print('Highest gain: {}'.format(highest_gain))

0.047028661304691965
Highest gain: education_num


### Decision Tree Model

#### Now we know how to make a single split. To construct an entire tree, we'll need to continue creating splits until the leaves only have a single class. We've been following the ID3 algorithm to construct decision trees. There are other algorithms like CART that use different measures for the split criterion. 

#### We'll use the ID3 Algorithm for constructing decision trees to accomplish this. This algorithm involves recursion and an understanding of time complexity. If you're unfamiliar with these topics, we suggest trying our Data Structures and Algorithms course. We also suggest learning about lambda functions through our command line course.

#### In general, recursion is the process of splitting a large problem into smaller chunks. Recursive functions will call themselves, then combine the results into a final output.

#### Building a tree is a perfect use case for recursive algorithms. At each node, we'll call a recursive function that will split the data into two branches. Each branch will lead to a node, and the function will call itself to build the tree out.

#### We wrote functions to calculate entropy and information gain. We've loaded these functions in as calc_entropy() and calc_information_gain().

#### Now we need a function that returns the name of the column we should use to split a data set. The function should take the name of the data set, the target column, and a list of columns we might want to split on as input.

#### Write a function named find_best_column() that returns the name of a column to split the data on. 

In [19]:
def find_best_column(data, target_name, columns):
    # Fill in the logic here to automatically find the column in columns to split on
    # data is a dataframe
    # target_name is the name of the target variable
    # columns is a list of potential columns to split on
    return None

#### Use find_best_column() to find the best column on which to split income.

In [20]:
# A list of columns to potentially split income with
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]
def find_best_column(data, target_name, columns):
    information_gains = []
    # Loop through and compute information gains
    for col in columns:
        information_gain = calc_information_gain(data, col, "high_income")
        information_gains.append(information_gain)

    # Find the name of the column with the highest gain
    highest_gain_index = information_gains.index(max(information_gains))
    highest_gain = columns[highest_gain_index]
    return highest_gain

income_split = find_best_column(income, "high_income", columns)
income_split

'education_num'

#### Let's build up to making the full id3() function by creating a simpler algorithm that we can extend. 

In [22]:
label_1s = []
label_0s = []
def id3(data, target, columns):
    unique_targets = pandas.unique(data[target])

    if len(unique_targets) == 1:
        if 0 in unique_targets:
            label_0s.append(0)
        elif 1 in unique_targets:
            label_1s.append(1)
        return
    
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    
    for split in [left_split, right_split]:
        id3(split, target, columns)

# Create the data set that we used in the example on the last screen
data = pandas.DataFrame([
    [0,20,0],
    [0,60,2],
    [0,40,1],
    [1,25,1],
    [1,35,2],
    [1,55,1]
    ])
# Assign column names to the data
data.columns = ["high_income", "age", "marital_status"]
# Call the function on our data to set the counters properly
id3(data, "high_income", ["age", "marital_status"])

#### Now we can store the entire tree, rather than the leaf labels only. We'll use nested dictionaries to do this. We can represent the root node with a dictionary, and branches with the keys left and right. We'll store the column we're splitting on as the key column, and the median value as the key median. Finally, we can store the label for a leaf as the key label. We'll also number each node as we go along using the number key.

#### We're now passing the tree dictionary into our id3 function and setting some keys on it. One complexity is in how we're creating the nested dictionary. For the left split, we're adding a key to the tree dictionary that looks like this:

- tree["left"] = {}

#### For the right side, we're adding:

- tree["right"] = {}

#### Now that we've added this key, we're able to pass our new dictionary into the recursive call to id3(). While this new dictionary will be the dictionary for that specific node, it will be tied back to the parent dictionary (because it's a key of the original dictionary).

#### This process will continue building up the nested dictionary. We'll be able to access the entire dictionary using the variable tree we define before the function. Think of each recursive call as building a piece of the tree, which we can then access after all of the functions have terminated.

In [23]:
# assign the correct label to the tree dictionary
# assign the column and median keys to the tree dictionary
# call the id3 function with the correct inputs: 
# id3(data, "high_income", ["age", "marital_status"], tree)
# Create a dictionary to hold the tree  
# It has to be outside of the function so we can access it later
tree = {}

# This list will let us number the nodes  
# It has to be a list so we can access it inside the function
nodes = []

def id3(data, target, columns, tree):
    unique_targets = pandas.unique(data[target])
    
    # Assign the number key to the node dictionary
    nodes.append(len(nodes) + 1)
    tree["number"] = nodes[-1]

    if len(unique_targets) == 1:
        # Insert code here that assigns the "label" field to the node dictionary
        return
    
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    
    # Insert code here that assigns the "column" and "median" fields to the node dictionary
    
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    split_dict = [["left", left_split], ["right", right_split]]
    
    for name, split in split_dict:
        tree[name] = {}
        id3(split, target, columns, tree[name])

# Call the function on our data to set the counters properly
id3(data, "high_income", ["age", "marital_status"], tree)
tree = {}
nodes = []

def id3(data, target, columns, tree):
    unique_targets = pandas.unique(data[target])
    nodes.append(len(nodes) + 1)
    tree["number"] = nodes[-1]

    if len(unique_targets) == 1:
        if 0 in unique_targets:
            tree["label"] = 0
        elif 1 in unique_targets:
            tree["label"] = 1
        return
    
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    
    tree["column"] = best_column
    tree["median"] = column_median
    
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    split_dict = [["left", left_split], ["right", right_split]]
    
    for name, split in split_dict:
        tree[name] = {}
        id3(split, target, columns, tree[name])


id3(data, "high_income", ["age", "marital_status"], tree)

In [24]:
tree

{'number': 1,
 'column': 'age',
 'median': 37.5,
 'left': {'number': 2,
  'column': 'age',
  'median': 25.0,
  'left': {'number': 3,
   'column': 'age',
   'median': 22.5,
   'left': {'number': 4, 'label': 0},
   'right': {'number': 5, 'label': 1}},
  'right': {'number': 6, 'label': 1}},
 'right': {'number': 7,
  'column': 'age',
  'median': 55.0,
  'left': {'number': 8,
   'column': 'age',
   'median': 47.5,
   'left': {'number': 9, 'label': 0},
   'right': {'number': 10, 'label': 1}},
  'right': {'number': 11, 'label': 0}}}

#### The tree dictionary shows all of the relevant information, but it doesn't look very nice. We can fix its appearance by printing it out in a nicer format.

#### To do this, we'll need to recursively iterate through our tree dictionary. Any dictionary that has a label key is a leaf. Whenever we find one, we'll print out the label. Otherwise, we'll loop through the tree's left and right keys and recursively call the same function.

#### We also need to keep track of a depth variable. This variable will allow us to use indentation to indicate the order of the nodes. Before we print anything out, we'll prefix it with the number of spaces corresponding to the depth variable.

In [25]:
def print_with_depth(string, depth):
    # Add space before a string
    prefix = "    " * depth
    # Print a string, and indent it appropriately
    print("{0}{1}".format(prefix, string))
    
    
def print_node(tree, depth):
    # Check for the presence of "label" in the tree
    if "label" in tree:
        # If found, then this is a leaf, so print it and return
        print_with_depth("Leaf: Label {0}".format(tree["label"]), depth)
        # This is critical -- without it, you'll get infinite recursion
        return
    # Print information about what the node is splitting on
    print_with_depth("{0} > {1}".format(tree["column"], tree["median"]), depth)
    
    # Create a list of tree branches
    branches = [tree["left"], tree["right"]]
        
    # Insert code here to recursively call print_node on each branch
    # Don't forget to increment depth when you pass it in

print_node(tree, 0)
def print_node(tree, depth):
    if "label" in tree:
        print_with_depth("Leaf: Label {0}".format(tree["label"]), depth)
        return
    print_with_depth("{0} > {1}".format(tree["column"], tree["median"]), depth)
    for branch in [tree["left"], tree["right"]]:
        print_node(branch, depth+1)

print_node(tree, 0)

age > 37.5
age > 37.5
    age > 25.0
        age > 22.5
            Leaf: Label 0
            Leaf: Label 1
        Leaf: Label 1
    age > 55.0
        age > 47.5
            Leaf: Label 0
            Leaf: Label 1
        Leaf: Label 0


#### Let's say we want to predict the following row:
    age    marital_status
    50     1
#### First, we'd split on age > 37.5 and go to the right. Then, we'd split on age > 55.0 and go to the left. Then, we'd split on age > 47.5 and go to the right. We'd end up predicting a 1 for high_income.

#### Making predictions with such a small tree is fairly straightforward, but what if we want to use the entire income dataframe? We wouldn't be able to eyeball predictions; we'd want an automated way to do this instead.

#### Let's write a function that makes predictions automatically. All we need to do is follow the split points we've already defined with a new row.

In [32]:
def predict(tree, row):
    if "label" in tree:
        return tree["label"]
    
    column = tree["column"]
    median = tree["median"]
    if row[column] <= median:
        return predict(tree["left"], row)
    else:
        return predict(tree["right"], row)

print(predict(tree, data.iloc[0]))

0


#### Now that we can make a prediction for a single row, we can write a function that makes predictions for multiple rows simultanously.

#### To do this, we'll use the apply() method on pandas dataframes to apply a function across each row. You can read more about the function in the pandas documentation. You'll need to pass in the axis=1 argument to apply the function to each row. This method will return a dataframe.

#### You can use the apply() method along with lambda functions to apply the predict() function to each row of new_data.

#### Create a function named batch_predict() that takes two parameters, tree and df.

#### It should use the apply() method to apply the predict() function across each row of df.

In [29]:
new_data = pandas.DataFrame([
    [40,0],
    [20,2],
    [80,1],
    [15,1],
    [27,2],
    [38,1]
    ])
# Assign column names to the data
new_data.columns = ["age", "marital_status"]

def batch_predict(tree, df):
    # Insert your code here
    pass

predictions = batch_predict(tree, new_data)
def batch_predict(tree, df):
    return df.apply(lambda x: predict(tree, x), axis=1)

predictions = batch_predict(tree, new_data)

In [31]:
predictions

0    0
1    0
2    0
3    0
4    1
5    0
dtype: int64

#### We learned how to create a full decision tree model, print the results, and use the tree to make predictions. We applied a modified version of the ID3 algorithm on a small data set for clarity.

#### Next, we'll learn about when to use decision trees, and how to use them most effectively. We can use the scikit-learn package to fit a decision tree. 

#### We use the DecisionTreeClassifier class for classification problems, and DecisionTreeRegressor for regression problems. The sklearn.tree package includes both of these classes.

#### In this case, we're predicting a binary outcome, so we'll use a classifier.

#### The first step is to train the classifier on the data. We'll use the fit method on a classifier to do this.

In [33]:
from sklearn.tree import DecisionTreeClassifier

# A list of columns to train with
# We've already converted all columns to numeric
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

# Instantiate the classifier
# Set random_state to 1 to make sure the results are consistent
clf = DecisionTreeClassifier(random_state=1)

# We've already loaded the variable "income," which contains all of the income data
clf.fit(income[columns], income["high_income"])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

#### Now that we've fit a model, we can make predictions. We'll want to split our data into training and testing sets first. If we don't, we'll be making predictions on the same data that we train our algorithm with. This leads to overfitting, and will make our error appear lower than it is.

#### We can avoid overfitting by always making predictions and evaluating error on data that we haven't trained our algorithm with. This will show us when we're overfitting by giving us a realistic error on data that the algorithm hasn't seen before.

#### We can split the data by shuffling the order of the dataframe, then selecting certain rows to include in the training set, and certain rows to include in the testing set.

#### In this case, we'll make 80% of our rows training data, and the rest testing data.

In [34]:
import numpy
import math

# Set a random seed so the shuffle is the same every time
numpy.random.seed(1)

# Shuffle the rows  
# This permutes the index randomly using numpy.random.permutation
# Then, it reindexes the dataframe with the result
# The net effect is to put the rows into random order
income = income.reindex(numpy.random.permutation(income.index))

train_max_row = math.floor(income.shape[0] * .8)
train = income.iloc[:train_max_row]
test = income.iloc[train_max_row:]

#### AUC ranges from 0 to 1, so it's ideal for binary classification. The higher the AUC, the more accurate our predictions.

#### We can compute AUC with the roc_auc_score function from sklearn.metrics. This function takes in two parameters:

- y_true: true labels
- y_score: predicted labels

#### It then calculates and returns the AUC value.

In [35]:
from sklearn.metrics import roc_auc_score

clf = DecisionTreeClassifier(random_state=1)
clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
error = roc_auc_score(test["high_income"], predictions)
print(error)

0.6971330661993492


#### The AUC for the predictions on the testing set is about .694. Let's compare this against the AUC for predictions on the training set to see if the model is overfitting.

#### It's normal for the model to predict the training set better than the testing set. After all, it has full knowledge of that data and the outcomes. However, if the AUC between training set predictions and actual values is significantly higher than the AUC between test set predictions and actual values, it's a sign that the model may be overfitting.

In [36]:
predictions = clf.predict(train[columns])
print(roc_auc_score(train["high_income"], predictions))

0.9471244501437455


### Overfittng

#### Our AUC on the training set was .947, and the AUC on the test set was .694. There's no hard and fast rule on when overfitting is occurring, but our model is predicting the training set much better than the test set. Splitting the data into training and testing sets doesn't prevent overfitting -- it just helps us detect and fix it.

#### Based on our AUC measurements, it appears that we are in fact overfitting.

#### Trees overfit when they have too much depth and make overly complex rules that match the training data, but aren't able to generalize well to new data. This may seem to be a strange principle at first, but the deeper a tree is, the worse it typically performs on new data.

#### There are three main ways to combat overfitting:

- "Prune" the tree after we build it to remove unnecessary leaves.
- Use ensembling to blend the predictions of many trees.
- Restrict the depth of the tree while we're building it.

#### While we'll explore all of these, we'll look at the third method first.

#### Limiting tree depth during the building process will result in more general rules. This prevents the tree from overfitting.

#### We can restrict tree depth by adding a few parameters when we initialize the DecisionTreeClassifier class:

- max_depth - Globally restricts how deep the tree can go
- min_samples_split - The minimum number of rows a node should have before it can be split; if this is set to 2, for example, then nodes with 2 rows won't be split, and will become leaves instead
- min_samples_leaf - The minimum number of rows a leaf must have
- min_weight_fraction_leaf - The fraction of input rows a leaf must have
- max_leaf_nodes - The maximum number of total leaves; this will cap the count of leaf nodes as the tree is being built

#### Some of these parameters aren't compatible, however. For example, we can't use max_depth and max_leaf_nodes together.

#### Now that we know what to tweak, let's improve our model.

In [37]:
# Set min_samples_split to 13 when creating the 
# DecisionTreeClassifier.
# Make predictions on the training set, compute the 
# AUC, and assign it to train_auc.
# Make predictions on the test set, compute the AUC, 
# and assign it to test_auc.
# Decision trees model from the last screen
clf = DecisionTreeClassifier(random_state=1)
clf = DecisionTreeClassifier(min_samples_split=13, random_state=1)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
print(train_auc)

0.7119952337831613
0.8426415759234083


#### By setting min_samples_split to 13, we managed to boost the test AUC from .694 to .700. The training set AUC decreased from .947 to .843, showing that the model we built was less overfit to the training set than before.

#### Let's play around with parameters some more.

In [38]:
# Set max_depth to 7 and min_samples_split to 13 when 
# creating the DecisionTreeClassifier.
# Make predictions on the training set, compute 
# the AUC, and assign it to train_auc.
# Make predictions on the test set, compute the 
# AUC, and assign it to test_auc.
# The first decision trees model we trained and tested
clf = DecisionTreeClassifier(random_state=1)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
print(train_auc)
clf = DecisionTreeClassifier(random_state=1, min_samples_split=13, max_depth=7)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
print(train_auc)

0.6971330661993492
0.9471244501437455
0.736437254165992
0.7423073733990373


#### We just improved the AUC again! The test set AUC increased to .744, while the training set AUC decreased to .748. We aren't overfitting anymore because both AUC values are about the same. Let's tweak the parameters more aggressively and see what happens. 

In [39]:
# Set max_depth to 2 and min_samples_split to 
# 100 when creating the DecisionTreeClassifier.
# Make predictions on the training set, compute the 
# AUC, and assign it to train_auc.
# Make predictions on the test set, compute the 
# AUC, and assign it to test_auc.
# The first decision tree model we trained and tested
clf = DecisionTreeClassifier(random_state=1)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
print(train_auc)
clf = DecisionTreeClassifier(random_state=1, min_samples_split=100, max_depth=2)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
print(train_auc)

0.6971330661993492
0.9471244501437455
0.6801979250344399
0.6916321420555511


### Underfitting

#### Our accuracy went down  relative to results before. This is because we're now underfitting. Underfitting is what occurs when our model is too simple to explain the relationships between the variables.

#### By artificially restricting the depth of our tree, we prevent it from creating a model that's complex enough to correctly categorize some of the rows. If we don't perform the artificial restrictions, however, the tree becomes too complex, fits quirks in the data that only exist in the training set, and doesn't generalize to new data.

#### This is known as the bias-variance tradeoff. Imagine that we take a random sample of training data and create many models. If the models' predictions for the same row are far apart from each other, we have high variance. Imagine this time that we take a random sample of the training data and create many models. If the models' predictions for the same row are close together but far from the actual value, then we have high bias.

#### High bias can cause underfitting -- if a model is consistently failing to predict the correct value, it may be that it's too simple to model the data faithfully.

#### High variance can cause overfitting. If a model varies its predictions significantly based on small changes in the input data, then it's likely fitting itself to quirks in the training data, rather than making a generalizable model.

#### We call this the bias-variance tradeoff because decreasing one characteristic will usually increase the other. This is a limitation of all machine learning algorithms.

#### Decision trees typically suffer from high variance. The entire structure of a decision tree can change if we make a minor alteration to its training data. By restricting the depth of the tree, we increase the bias and decrease the variance. If we restrict the depth too much, we increase bias to the point where it will underfit.

#### We can induce variance and see what happens with a decision tree. To introduce noise into the data, we'll add a column of random values. A model with high variance (like a decision tree) will pick up on this noise, and overfit to it. This is because models with high variance are very sensitive to small changes in input data.

In [40]:
numpy.random.seed(1)

# Generate a column containing random numbers from 0 to 4
income["noise"] = numpy.random.randint(4, size=income.shape[0])

# Adjust "columns" to include the noise column
columns = ["noise", "age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

# Make new train and test sets
train_max_row = math.floor(income.shape[0] * .8)
train = income.iloc[:train_max_row]
test = income.iloc[train_max_row:]

# Initialize the classifier
clf = DecisionTreeClassifier(random_state=1)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
print(train_auc)

0.7033070734546627
0.9750761614350801


#### The random noise column causes significant overfitting. Our test set accuracy decreases to .703, and our training set accuracy increases to .975.

#### One way to prevent overfitting is to block the tree from growing beyond a certain depth (we tried this before). Another technique is called pruning. Pruning involves building a full tree, and then removing the leaves that don't add to prediction accuracy. Pruning prevents a model from becoming overly complex. It can result in a simpler model that has higher accuracy on the testing set.

### Conclusion

#### Let's go over the main advantages and disadvantages of using decision trees. The main advantages of using decision trees is that they're:

- Easy to interpret
- Relatively fast to fit and make predictions
- Able to handle multiple types of data
- Able to pick up nonlinearities in data, and usually fairly accurate

#### The main disadvantage of using decision trees is their tendency to overfit.

#### Decision trees are a good choice for tasks where it's important to be able to interpret and convey why the algorithm is doing what it's doing.

#### The most powerful way to reduce decision tree overfitting is to create ensembles of trees. The random forest algorithm is a popular choice for doing this. In cases where prediction accuracy is the most important consideration, random forests usually perform better.